# Cross Validation

Finalmente vamos a realizar cross validation para los modelos basados en un Random Forest y Deep Neural Network. 

### Importaciones y preprocessing

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import  recall_score, precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Recall, Precision
import random

In [2]:
import sys
import os

sys.path.append(os.path.abspath(".."))


from src.preprocessing import load_data, split_features_target, get_column_types, build_preprocessor, prepare_features

df = load_data("C:/Users/Sebastian/Desktop/ProyectoML/bank.csv")

X, y, preprocessor = prepare_features(df, 'y') 

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = 0.2,
    stratify=y,
    random_state=42
)

### Reproducibilidad

In [3]:
SEED = 42

os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

### Creamos Folds Estratificados y entrenamos

In [4]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)


early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-5
)

In [5]:
rf_precision = []
rf_recall = []
dnn_precision = []
dnn_recall = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"\nFold {fold}")
    
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    X_train_proc = preprocessor.fit_transform(X_train)
    X_val_proc = preprocessor.transform(X_val)  

    threshold = 0.5

    #random_forest

    rf = RandomForestClassifier(
        n_estimators=200,
        max_depth=6,
        min_samples_split = 10,
        min_samples_leaf = 5,
        max_features = 'sqrt',
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    )

    rf.fit(X_train_proc, y_train)


    y_prob_rf = rf.predict_proba(X_val_proc)[:, 1]
    y_pred_rf = (y_prob_rf >= threshold).astype(int)
    precision_rf = precision_score(y_val, y_pred_rf, pos_label=1)
    recall_rf = recall_score(y_val, y_pred_rf, pos_label=1)

    rf_precision.append(precision_rf)
    rf_recall.append(recall_rf)
    
    #DNN
    
    model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_proc.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, activation='relu'),
    Dropout(0.3),

    Dense(32, activation='relu'),

    Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', Recall(name='recall'), Precision(name='precision')]
    )

    classes = np.unique(y_train)
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=classes,
        y=y_train
    )

    class_weight_dict = dict(zip(classes, class_weights))

    history = model.fit(
        X_train_proc, y_train,
        validation_split=0.2,
        epochs=100,
        batch_size=32,
        class_weight=class_weight_dict,
        callbacks=[early_stop, reduce_lr],
        verbose=0
    )

    y_pred_prob = model.predict(X_val_proc)
    y_pred = (y_pred_prob >= threshold).astype(int)
    
    precision = precision_score(y_val, y_pred, pos_label=1)
    recall = recall_score(y_val, y_pred, pos_label=1)

    dnn_precision.append(precision)
    dnn_recall.append(recall)


Fold 1


c:\Users\Sebastian\Desktop\ProyectoML\venv\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

Fold 2


c:\Users\Sebastian\Desktop\ProyectoML\venv\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

Fold 3


...

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

Fold 4


c:\Users\Sebastian\Desktop\ProyectoML\venv\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

Fold 5


c:\Users\Sebastian\Desktop\ProyectoML\venv\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


### Visualización de resultados

In [6]:
results = pd.DataFrame({
    'Modelo': ['Random Forest', 'DNN'],
    
    'Precision (mean)': [
        np.mean(rf_precision),
        np.mean(dnn_precision)
    ],
    'Precision (std)': [
        np.std(rf_precision),
        np.std(dnn_precision)
    ],
    
    'Recall (mean)': [
        np.mean(rf_recall),
        np.mean(dnn_recall)
    ],
    'Recall (std)': [
        np.std(rf_recall),
        np.std(dnn_recall)
    ]
})

results

,Modelo,Precision (mean),Precision (std),Recall (mean),Recall (std)
0,Random Forest,0.388686,0.019594,0.771758,0.048095
1,DNN,0.318440,0.051609,0.827473,0.076587


# **Final Conclusions**

The cross-validation results show a clear trade-off between the evaluated models.

The **Random Forest** model achieves a higher **precision**, indicating that its positive predictions (class 1) are more reliable and generate fewer false positives. Additionally, its lower standard deviation suggests more stable performance across folds.

On the other hand, the **Deep Neural Network (DNN)** obtains a higher **recall**, meaning it is able to identify a larger proportion of actual positive cases. This comes at the cost of lower precision, as the model tends to classify more instances as positive.

Given the objective of the problem — identifying potential clients who may subscribe to the service — **recall is considered more critical than precision**. Missing a potential client (false negative) may represent a lost business opportunity, while false positives are less costly, as some external factors (e.g., sales interaction or customer decisions) are not captured by the dataset.

Therefore, despite its lower precision, the **DNN is selected as the preferred model**, as it maximizes the detection of potential customers.